### About
#### Create a unique hash ID for each QnA pair.

### Import necessary libraries and packages

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import io
import requests
from tqdm.notebook import tqdm
import subprocess
import time
import json

### Get data

In [ ]:
data = 'https://raw.githubusercontent.com/hariprasath-v/Nnet101_Assistant/refs/heads/main/data/Stackoverflow_data(neural_networks_stats)_pre_processed_Gemini_LLM.csv'
data = pd.read_csv(data)
data.head()

,question,q_link,tags,q_question_id,q_is_answered,q_accepted_answer_id,q_view_count,q_answer_count,q_score,q_last_activity_date,q_creation_date,a_score,a_creation_date,a_answer,answer
0,How to choose the number of hidden layers and ...,https://stats.stackexchange.com/questions/181/...,model-selection|neural-networks,181,True,1097,1145801,10,820,1661947755,1279584902,671,1280715630,"I realize this question has been answered, but...",**Network Configuration in Neural Networks**\n...
1,What should I do when my neural network doesn&...,https://stats.stackexchange.com/questions/3520...,neural-networks|faq,352036,True,352037,365434,9,368,1701358003,1529367960,455,1529367960,1. Verify that your code is bug free\nThere's...,**Key Considerations for Neural Network Develo...
2,"What exactly are keys, queries, and values in ...",https://stats.stackexchange.com/questions/4219...,neural-networks|natural-language|attention|mac...,421935,True,424127,261109,11,309,1708928023,1565686855,281,1567068576,The key/value/query formulation of attention i...,In the key/value/query formulation of attentio...
3,What is batch size in neural network?,https://stats.stackexchange.com/questions/1535...,neural-networks|python|terminology|keras,153531,True,153535,731148,6,305,1650529048,1432286121,421,1432288067,The batch size defines the number of samples t...,**Summary**\n\n**Batch Size**\n\nBatch size de...
4,What are the advantages of ReLU over sigmoid f...,https://stats.stackexchange.com/questions/1262...,machine-learning|neural-networks|sigmoid-curve...,126238,True,126362,290897,9,234,1723495231,1417486429,205,1417567286,Two additional major benefits of ReLUs are spa...,**Summary:**\n\nRectified Linear Units (ReLUs)...


In [3]:
data.shape

(500, 15)

In [4]:
data_dict = data[['question','tags','answer']].to_dict(orient='records')

### Generate unique ID

In [5]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['tags']}-{doc['question']}-{doc['answer'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [6]:
for doc in data_dict:
    doc['id'] = generate_document_id(doc)

In [7]:
len(set([ids['id'] for ids in data_dict]))

500

In [8]:
data_dict[2]

{'question': 'What exactly are keys, queries, and values in attention mechanisms?',
 'tags': 'neural-networks|natural-language|attention|machine-translation',
 'answer': "In the key/value/query formulation of attention, the query represents the input to be matched, while the keys and values store the candidate matches and their associated information.\nLike a search engine, attention's query is matched against a set of keys, and the best matched values are returned.\nThis retrieval process involves calculating a probability vector alpha, which determines the proportion of each value to include in the output.\n\nThe first paper (Bahdanau et al. 2015) calculates alpha using a neural network, which is computationally expensive.\nThe second paper (Vaswani et al. 2017) proposes a more efficient approach, where the keys and values are first projected onto a common space, and then a similarity measure is used to calculate alpha.\nThis approach corresponds to the retrieval system concept, wher

In [9]:
from collections import defaultdict

In [10]:
hashes = defaultdict(list)

for doc in data_dict:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [11]:
len(hashes), len(data_dict)

(500, 500)

In [13]:
with open('/workspaces/Nnet101_Assistant/data/nnet_101_qna_with_id.json', 'wt') as f_out:
    json.dump(data_dict, f_out, indent=2)

In [15]:
!head /workspaces/Nnet101_Assistant/data/nnet_101_qna_with_id.json

[
  {
    "question": "How to choose the number of hidden layers and nodes in a feedforward neural network?",
    "tags": "model-selection|neural-networks",
    "answer": "**Network Configuration in Neural Networks**\n\n**Standardization**\nThere is no single standardized method for configuring networks. However, guidelines exist for setting the number and type of network layers, as well as the number of neurons in each layer.\n\n**Initial Architecture Setup**\nBy following specific rules, one can establish a competent network architecture. This involves determining the number and type of neuronal layers and the number of neurons within each layer. This approach provides a foundational architecture but may not be optimal.\n\n**Iterative Tuning**\nOnce the network is initialized, its configuration can be iteratively tuned during training. Ancillary algorithms, such as pruning, can be used to eliminate unnecessary nodes, optimizing the network's size and performance.\n\n**Network Layer T